In [1]:
from types import SimpleNamespace
from collections import Counter
import os
import re
import pathlib
import array
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import math

In [2]:
DATASET_VERSION = 'ca-100'
COMPETITION_ROOT = '../input/vectors3'
DATASET_ROOT = f'../input/text-preprocessing/data/{DATASET_VERSION}'
WORKING_ROOT = f'data/{DATASET_VERSION}'
DATASET_PREFIX = 'ca.wiki'

In [3]:
params = SimpleNamespace(
    embedding_dim = 256,
    window_size = 7,
    batch_size = 2048,
    epochs = 4,
    preprocessed = f'{DATASET_ROOT}/{DATASET_PREFIX}',
    working = f'{WORKING_ROOT}/{DATASET_PREFIX}',
    modelname = f'{WORKING_ROOT}/{DATASET_VERSION}.pt',
    train = True
)

In [4]:
class Vocabulary(object):
    def __init__(self, pad_token='<pad>', unk_token='<unk>', eos_token='<eos>'):
        self.token2idx = {}
        self.idx2token = []
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_token = eos_token
        if pad_token is not None:
            self.pad_index = self.add_token(pad_token)
        if unk_token is not None:
            self.unk_index = self.add_token(unk_token)
        if eos_token is not None:
            self.eos_index = self.add_token(eos_token)

    def add_token(self, token):
        if token not in self.token2idx:
            self.idx2token.append(token)
            self.token2idx[token] = len(self.idx2token) - 1
        return self.token2idx[token]

    def get_index(self, token):
        if isinstance(token, str):
            return self.token2idx.get(token, self.unk_index)
        else:
            return [self.token2idx.get(t, self.unk_index) for t in token]

    def get_token(self, index):
        return self.idx2token[index]

    def __len__(self):
        return len(self.idx2token)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.__dict__.update(pickle.load(f))

In [5]:
def batch_generator(idata, target, batch_size, shuffle=True):
    nsamples = len(idata)
    if shuffle:
        perm = np.random.permutation(nsamples)
    else:
        perm = range(nsamples)

    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            yield idata[batch_idx], target[batch_idx]
        else:
            yield idata[batch_idx], None

In [6]:
def load_preprocessed_dataset(prefix):
    # Try loading precomputed vocabulary and preprocessed data files
    token_vocab = Vocabulary()
    token_vocab.load(f'{prefix}.vocab')
    data = []
    for part in ['train', 'valid', 'test']:
        with np.load(f'{prefix}.{part}.npz') as set_data:
            idata, target = set_data['idata'], set_data['target']
            data.append((idata, target))
            print(f'Number of samples ({part}): {len(target)}')
    print("Using precomputed vocabulary and data files")
    print(f'Vocabulary size: {len(token_vocab)}')
    return token_vocab, data

In [7]:
def train(model, criterion, optimizer, idata, target, batch_size, device, log=False):
    model.train()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    for X, y in batch_generator(idata, target, batch_size, shuffle=True):
        # Get input and target sequences from batch
        X = torch.tensor(X, dtype=torch.long, device=device)
        y = torch.tensor(y, dtype=torch.long, device=device)

        model.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        # Training statistics
        total_loss += loss.item()
        ncorrect += (torch.max(output, 1)[1] == y).sum().item()
        ntokens += y.numel()
        niterations += 1
        if niterations == 200 or niterations == 500 or niterations % 1000 == 0:
            print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={100*ncorrect/ntokens:.1f}, loss={total_loss/ntokens:.2f}')

    total_loss = total_loss / ntokens
    accuracy = 100 * ncorrect / ntokens
    if log:
        print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={accuracy:.1f}, loss={total_loss:.2f}')
    return accuracy, total_loss

In [8]:
def validate(model, criterion, idata, target, batch_size, device):
    model.eval()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    y_pred = []
    with torch.no_grad():
        for X, y in batch_generator(idata, target, batch_size, shuffle=False):
            # Get input and target sequences from batch
            X = torch.tensor(X, dtype=torch.long, device=device)
            output = model(X)
            if target is not None:
                y = torch.tensor(y, dtype=torch.long, device=device)
                loss = criterion(output, y)
                total_loss += loss.item()
                ncorrect += (torch.max(output, 1)[1] == y).sum().item()
                ntokens += y.numel()
                niterations += 1
            else:
                pred = torch.max(output, 1)[1].detach().to('cpu').numpy()
                y_pred.append(pred)

    if target is not None:
        total_loss = total_loss / ntokens
        accuracy = 100 * ncorrect / ntokens
        return accuracy, total_loss
    else:
        return np.concatenate(y_pred)

In [9]:
# Create working dir
pathlib.Path(WORKING_ROOT).mkdir(parents=True, exist_ok=True)

In [10]:
# Select device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("WARNING: Training without GPU can be very slow!")

In [11]:
vocab, data = load_preprocessed_dataset(params.preprocessed)

Number of samples (train): 82284341
Number of samples (valid): 164765
Number of samples (test): 165837
Using precomputed vocabulary and data files
Vocabulary size: 100002


In [12]:
# 'El Periodico' validation dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_valid.csv')
tokens = valid_x_df.columns[1:]
valid_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')
valid_y_df = pd.read_csv(f'{COMPETITION_ROOT}/y_valid.csv')
valid_y = valid_y_df['token'].apply(vocab.get_index).to_numpy(dtype='int32')

In [13]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -Inf)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

The **xavier_normal_()** initialization function fills the input Tensor with values according to the method described in **Understanding the difficulty of training deep feedforward neural networks** - Glorot, X. & Bengio, Y. (2010), using a uniform distribution. The resulting tensor will have values sampled from a uniform distribution \\(\mathcal{U}(-a, a)\\) where

\\(    a = \text{gain} \times \sqrt{\frac{6}{\text{fan\_in} + \text{fan\_out}}}\\)

Also known as Glorot initialization.

In [14]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, bias=True):
        super().__init__()
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        # Empirically observed the convergence to be much better with the scaled initialization
        nn.init.xavier_uniform_(self.k_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.v_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.q_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.out_proj.weight)
        if self.out_proj.bias is not None:
            nn.init.constant_(self.out_proj.bias, 0.)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    def forward(self, x):
        # x shape is (B, W, E)
        q = self.q_proj(x)
        # q shape is (B, W, E)
        k = self.k_proj(x)
        # k shape is (B, W, E)
        v = self.v_proj(x)
        # k shape is (B, W, E)
        y, _ = attention(q, k, v)
        # y shape is (B, W, E)
        y = self.out_proj(y)
        # y shape is (B, W, E)
        return y

In [15]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, dim_feedforward=512, dropout=0.1, activation="relu"):
        super().__init__()
        self.self_attn = SelfAttention(d_model)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

In [16]:
class Predictor(nn.Module):
    ''' Baseline model'''
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        self.att = TransformerLayer(embedding_dim)
        self.position_embedding = nn.Parameter(torch.Tensor(context_words, embedding_dim))
        nn.init.xavier_uniform_(self.position_embedding)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = e + self.position_embedding
        # u shape is (B, W, E)
        v = self.att(u)
        # v shape is (B, W, E)
        x = v.sum(dim=1)
        # x shape is (B, E)
        y = self.lin(x)
        # y shape is (B, V)
        return y

    
class Predictor_3attention(nn.Module):
    ''' 3 concatenated TransformerLayers'''
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        self.att1 = TransformerLayer(embedding_dim)
        self.att2 = TransformerLayer(embedding_dim)
        self.att3 = TransformerLayer(embedding_dim)
        self.position_embedding = nn.Parameter(torch.Tensor(context_words, embedding_dim))
        nn.init.xavier_uniform_(self.position_embedding)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = e + self.position_embedding
        # u shape is (B, W, E)
        v = self.att1(u)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        x = torch.sum(v,dim=1)
        # x shape is (B, E)
        y = self.lin(x)
        # y shape is (B, V)
        return y

In [17]:
class CosinePositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        """
        Args:
            d_model (int): Embedding dimensionality
            dropout (float): Dropout probability on the forward pass
            max_len (int): Maximum length of a sequence to expect
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        # Don the transpose because the class originally expected shape [seq_len, batch_size, embedding_dim]
        x = torch.transpose(x, 0, 1)
        x = x + self.pe[:x.size(0)]
        return torch.transpose(self.dropout(x), 0 ,1) # Return original size


class Predictor_3attention_cosine_encoding(nn.Module):
    ''' 3 concatenated TransformerLayers with Cosine positional embedding'''
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        self.att1 = TransformerLayer(embedding_dim)
        self.att2 = TransformerLayer(embedding_dim)
        self.att3 = TransformerLayer(embedding_dim)
        self.position_embedding = CosinePositionalEncoding(embedding_dim)
        #nn.init.xavier_uniform_(self.position_embedding)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = self.position_embedding(e)
        # u shape is (B, W, E)
        v = self.att1(u)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        x = torch.sum(v,dim=1)
        # x shape is (B, E)
        y = self.lin(x)
        # y shape is (B, V)
        return y

In [18]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model,n_heads, dim_feedforward=512, dropout=0.1, activation="relu"):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = int(d_model/n_heads)
        self.att_heads = nn.ModuleList([TransformerLayer(self.d_head) for i in range(n_heads)])

    def forward(self, src):
        out = None
        for i,emb_section in enumerate(torch.split(src,self.d_head,dim = 2)):
            #print(emb_section.shape,src.shape)
            out_section = self.att_heads[i](emb_section)
            if out is None:
                out = out_section
            else:
                out = torch.cat((out,out_section),2)
        return out


class Predictor_3multihead_attention(nn.Module):
    ''' Model with 3 concatenated multihead attention blocks'''
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1, n_heads=2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        self.att1 = MultiHeadAttention(embedding_dim, n_heads)
        self.position_embedding = nn.Parameter(torch.Tensor(context_words, embedding_dim))
        nn.init.xavier_uniform_(self.position_embedding)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = e + self.position_embedding
        # u shape is (B, W, E)
        v = self.att1(u)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        v = self.att2(v)
        # v shape is (B, W, E)
        x = torch.sum(v,dim=1)
        # x shape is (B, E)
        y = self.lin(x)
        # y shape is (B, V)
        return y

In [19]:
class EncoderLayer(nn.Module):

    def __init__(self, embed_dim, ffn_dim, num_heads, dropout=0.0):
        """
        Args:
            embed_dim (int): Embedding dimensionality (input, output & self-attention)
            ffn_dim (int): Inner dimensionality in the FFN
            num_heads (int): Number of heads of the multi-head attention block
            dropout (float): Dropout probability
        """
        super(EncoderLayer, self).__init__()

        self.att1 = MultiHeadAttention(embed_dim, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ffn_dim),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(ffn_dim, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Multihead Attention block
        selfattn_out = self.att1(x)

        # Add + normalize block (1)
        x = self.norm1(x + selfattn_out)

        # FFN block
        ffn_out = self.ffn(x)
        ffn_out = self.dropout(ffn_out)

        # Add + normalize block (2)
        x = self.norm2(x + ffn_out)

        return x



class Predictor_3multihead_attention_fcn_norm(nn.Module):
    ''' Concatenation of Transformer encoder layers with multihead attentions'''
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1, num_layers=6, n_heads=2, ffn_dim= 2048):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)        
        self.position_embedding = nn.Parameter(torch.Tensor(context_words, embedding_dim))
        nn.init.xavier_uniform_(self.position_embedding)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(embedding_dim, ffn_dim, n_heads, dropout = 0.1)
            for _ in range(num_layers)
        ])

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = e + self.position_embedding
        # u shape is (B, W, E)
        for layer in self.encoder_layers:
            u = layer(u)
        # v shape is (B, W, E)
        x = torch.sum(u,dim=1)
        # x shape is (B, E)
        y = self.lin(x)
        # y shape is (B, V)
        return y

In [20]:
#model = Predictor(len(vocab), params.embedding_dim).to(device)
#model = Predictor_3attention(len(vocab), params.embedding_dim).to(device)
#model = Predictor_3attention_cosine_encoding(len(vocab), params.embedding_dim).to(device)
model = Predictor_3multihead_attention_fcn_norm(len(vocab), params.embedding_dim, n_heads = 4, num_layers=3).to(device)

In [21]:
print(model)
for name, param in model.named_parameters():
    print(f'{name:20} {param.numel()} {list(param.shape)}')
print(f'TOTAL                {sum(p.numel() for p in model.parameters())}')

Predictor_3multihead_attention_fcn_norm(
  (emb): Embedding(100002, 256, padding_idx=0)
  (lin): Linear(in_features=256, out_features=100002, bias=False)
  (encoder_layers): ModuleList(
    (0): EncoderLayer(
      (att1): MultiHeadAttention(
        (att_heads): ModuleList(
          (0): TransformerLayer(
            (self_attn): SelfAttention(
              (k_proj): Linear(in_features=64, out_features=64, bias=True)
              (v_proj): Linear(in_features=64, out_features=64, bias=True)
              (q_proj): Linear(in_features=64, out_features=64, bias=True)
              (out_proj): Linear(in_features=64, out_features=64, bias=True)
            )
            (linear1): Linear(in_features=64, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=512, out_features=64, bias=True)
            (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((64,), eps=1e-05, elementwi

In [22]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(reduction='sum')

train_accuracy = []
wiki_accuracy = []
valid_accuracy = []
for epoch in range(params.epochs+1):
    acc, loss = train(model, criterion, optimizer, data[0][0], data[0][1], params.batch_size, device, log=True)
    train_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | train accuracy={acc:.1f}%, train loss={loss:.2f}')
    acc, loss = validate(model, criterion, data[1][0], data[1][1], params.batch_size, device)
    wiki_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (wikipedia)')
    acc, loss = validate(model, criterion, valid_x, valid_y, params.batch_size, device)
    valid_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (El Periódico)')

# Save model
torch.save(model.state_dict(), params.modelname)

Train: wpb=2048, num_updates=200, accuracy=5.2, loss=8.35
Train: wpb=2048, num_updates=500, accuracy=10.6, loss=7.45
Train: wpb=2048, num_updates=1000, accuracy=19.3, loss=6.37
Train: wpb=2048, num_updates=2000, accuracy=26.4, loss=5.52
Train: wpb=2048, num_updates=3000, accuracy=29.7, loss=5.11
Train: wpb=2048, num_updates=4000, accuracy=31.9, loss=4.86
Train: wpb=2048, num_updates=5000, accuracy=33.4, loss=4.68
Train: wpb=2048, num_updates=6000, accuracy=34.5, loss=4.54
Train: wpb=2048, num_updates=7000, accuracy=35.5, loss=4.43
Train: wpb=2048, num_updates=8000, accuracy=36.2, loss=4.33
Train: wpb=2048, num_updates=9000, accuracy=36.9, loss=4.26
Train: wpb=2048, num_updates=10000, accuracy=37.4, loss=4.19
Train: wpb=2048, num_updates=11000, accuracy=37.9, loss=4.13
Train: wpb=2048, num_updates=12000, accuracy=38.4, loss=4.08
Train: wpb=2048, num_updates=13000, accuracy=38.7, loss=4.03
Train: wpb=2048, num_updates=14000, accuracy=39.1, loss=3.99
Train: wpb=2048, num_updates=15000, ac

In [23]:
# 'El Periodico' test dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_test.csv')
test_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')
y_pred = validate(model, None, test_x, None, params.batch_size, device)
y_token = [vocab.get_token(index) for index in y_pred]

In [24]:
submission = pd.DataFrame({'id':valid_x_df['id'], 'token': y_token}, columns=['id', 'token'])
print(submission.head())
submission.to_csv('submission.csv', index=False)

   id  token
0   0      l
1   1      s
2   2  haver
3   3     no
4   4     el
